In [1]:
import pandas as pd
import csv
import os

import linecache
import sys

import matplotlib.pyplot as plt
import math
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

#from prettytable import PrettyTable

#簡轉繁用 
#https://pypi.python.org/pypi/hanziconv/0.2.1
from hanziconv import HanziConv

import matplotlib_venn 
from matplotlib_venn import venn2, venn2_circles


import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from numpy import mean, sqrt, square, arange

import pickle

import random
import datetime
import operator

#https://stackoverflow.com/questions/14519177/python-exception-handling-line-number
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))


In [31]:
# READ TAIWAN DATA WITH FEATURES
resampled_Real_data_df = pd.read_csv('./NameData/Newresampled_Real_data_df.csv', parse_dates = True)
resampled_FB_data_df = pd.read_csv('./NameData/Newresampled_FB_data_df.csv', parse_dates = True)

In [10]:
# READ TAIWAN DATA WITH FEATURES
taiwandata = pd.read_csv('./NameData/EnglishTaiwanFeatureData.csv', parse_dates = True)
resample_taiwandata = pd.read_csv('./NameData/EnglishResample_TaiwanFeatureData.csv', parse_dates = True)

In [17]:
resample_taiwandata = resample_taiwandata.rename(columns = {"E_Name": "name", "Gender_index": "GuessedGender", 
                                                            "EN_Surname": "LastName","EN_Firstname": "FirstName", 
                                                            "YearRange": "BirthYear", "Year": "message"})

In [18]:
resample_taiwandata.head()

,C_Name,name,Gender,message,LastName,FirstName,Age,GuessedGender,FirstName1,FirstName2,...,FN2,FN1_sonin,FN1_muin,FN2_sonin,FN2_muin,FN1_sonin_index,FN2_sonin_index,FN1_muin_index,FN2_muin_index,BirthYear
0,紀明德,Ming-Te Chi,M,1978,Chi,Ming Te,41,1,Ming,Te,...,8830,M,ing,T,e,55,57,22,142,7
1,胡毓忠,Yuh-Jong Hu,M,1958,Hu,Yuh Jong,61,1,Yuh,Jong,...,8844,Y,uh,J,ong,56,61,168,23,3
2,郭桐惟,Tung-Wei Kuo,M,1987,Kuo,Tung Wei,32,1,Tung,Wei,...,8639,T,ung,W,ei,57,59,169,60,9
3,張宏慶,Hung-Ching Jang,M,1962,Jang,Hung Ching,57,1,Hung,Ching,...,8620,H,ung,Ch,ing,58,62,169,22,4
4,廖文宏,Wen-Hung Liao,M,1965,Liao,Wen Hung,54,1,Wen,Hung,...,8581,W,en,H,ung,59,58,76,169,5


# Resample Column

In [32]:
resampled_Real_data_df = resampled_Real_data_df.drop('FN2_radical',1)
resampled_Real_data_df = resampled_Real_data_df.drop('FN1_radical',1)
resampled_Real_data_df = resampled_Real_data_df.drop('FN2_radical_index',1)
resampled_Real_data_df = resampled_Real_data_df.drop('FN1_radical_index',1)
resampled_Real_data_df = resampled_Real_data_df.drop('FN1_in',1)
resampled_Real_data_df = resampled_Real_data_df.drop('FN2_in',1)
resampled_Real_data_df = resampled_Real_data_df.drop('FN1_muin',1)
resampled_Real_data_df = resampled_Real_data_df.drop('FN2_muin',1)
resampled_Real_data_df = resampled_Real_data_df.drop('天格',1)
resampled_Real_data_df = resampled_Real_data_df.drop('人格',1)
resampled_Real_data_df = resampled_Real_data_df.drop('地格',1)
resampled_Real_data_df = resampled_Real_data_df.drop('外格',1)
resampled_Real_data_df = resampled_Real_data_df.drop('總格',1)
resampled_Real_data_df = resampled_Real_data_df.drop('三才',1)
resampled_Real_data_df = resampled_Real_data_df.drop('天格_Structure',1)
resampled_Real_data_df = resampled_Real_data_df.drop('人格_Structure',1)
resampled_Real_data_df = resampled_Real_data_df.drop('地格_Structure',1)
resampled_Real_data_df = resampled_Real_data_df.drop('外格_Structure',1)
resampled_Real_data_df = resampled_Real_data_df.drop('總格_Structure',1)
resampled_Real_data_df = resampled_Real_data_df.drop('三才_State',1)

In [39]:
# Only use 'mu' without then 'in'
resampled_Real_data_df['FN1_muin'] = resampled_Real_data_df['FN1_mu_in']
resampled_Real_data_df['FN2_muin'] = resampled_Real_data_df['FN2_mu_in']
resampled_Real_data_df = resampled_Real_data_df.drop('FN1_mu_in',1)
resampled_Real_data_df = resampled_Real_data_df.drop('FN2_mu_in',1)

In [110]:
# Re-generate muin_index by using new muin

with open('./util_files/mu_in_list', 'rb') as handle:
    mu_in_list = pickle.loads(handle.read())
    
def add_pin_in_index_column(pin_yin, mode):
    #mode 1 = sonin
    #mode 2 = muin
    if pin_yin == None:
        return -1
    try:
        if mode == 'muin':
            if pin_yin in mu_in_list:
                return mu_in_list.index(pin_yin)
            else:
                mu_in_list.append(pin_yin)
                return mu_in_list.index(pin_yin)

        if mode == 'sonin':
            if pin_yin in son_in_list:
                return son_in_list.index(pin_yin)
            else:
                son_in_list.append(pin_yin)
                return son_in_list.index(pin_yin)

    except Exception as e:
        print (e)
        print (fileName)
        PrintException() 
        
resampled_Real_data_df['FN1_muin_index'] = resampled_Real_data_df.FN1_muin.apply(lambda x: add_pin_in_index_column(x,'muin'))
resampled_Real_data_df['FN2_muin_index'] = resampled_Real_data_df.FN2_muin.apply(lambda x: add_pin_in_index_column(x,'muin'))

resampled_FB_data_df['FN1_muin_index'] = resampled_FB_data_df.FN1_muin.apply(lambda x: add_pin_in_index_column(x,'muin'))
resampled_FB_data_df['FN2_muin_index'] = resampled_FB_data_df.FN2_muin.apply(lambda x: add_pin_in_index_column(x,'muin'))


In [130]:
resampled_Real_data_df['FirstName1'] = resampled_Real_data_df.FirstName.apply(lambda x:x[0])
resampled_Real_data_df['FirstName2'] = resampled_Real_data_df.FirstName.apply(lambda x:x[1])

resampled_FB_data_df['FirstName1'] = resampled_FB_data_df.FirstName.apply(lambda x:x[0])
resampled_FB_data_df['FirstName2'] = resampled_FB_data_df.FirstName.apply(lambda x:x[1])

In [131]:
resampled_Real_data_df.head()

,FirstName,LastName,gender,message,name,FN1,FN2,FN1_sonin,FN2_sonin,FN1_sonin_index,FN2_sonin_index,FN1_muin_index,FN2_muin_index,BirthYear,GuessedGender,BFN12P,FirstName1,FirstName2,FN1_muin,FN2_muin
0,秀珠,吳,NaN,1947,吳秀珠,70,596,x,zh,16,8,62,160,1,0,1,秀,珠,iu,u
1,豐博,林,NaN,1945,林豐博,277,72,f,b,13,7,25,154,1,1,6,豐,博,eng,o
2,義勝,康,1.0,1945,康義勝,650,156,y,sh,3,5,164,25,1,1,1,義,勝,i,eng
3,麗,吳,0.0,1946,吳麗,8578,203,-1,l,-1,18,230,164,1,0,6,,麗,-1,i
4,淡浪,張,NaN,1948,張淡浪,1008,104,d,l,1,18,80,27,1,1,6,淡,浪,an,ang


In [34]:
resampled_FB_data_df = resampled_FB_data_df.drop('FN2_radical',1)
resampled_FB_data_df = resampled_FB_data_df.drop('FN1_radical',1)
resampled_FB_data_df = resampled_FB_data_df.drop('FN2_radical_index',1)
resampled_FB_data_df = resampled_FB_data_df.drop('FN1_radical_index',1)
resampled_FB_data_df = resampled_FB_data_df.drop('FN1_in',1)
resampled_FB_data_df = resampled_FB_data_df.drop('FN2_in',1)
resampled_FB_data_df = resampled_FB_data_df.drop('FN1_muin',1)
resampled_FB_data_df = resampled_FB_data_df.drop('FN2_muin',1)
resampled_FB_data_df = resampled_FB_data_df.drop('天格',1)
resampled_FB_data_df = resampled_FB_data_df.drop('人格',1)
resampled_FB_data_df = resampled_FB_data_df.drop('地格',1)
resampled_FB_data_df = resampled_FB_data_df.drop('外格',1)
resampled_FB_data_df = resampled_FB_data_df.drop('總格',1)
resampled_FB_data_df = resampled_FB_data_df.drop('三才',1)
resampled_FB_data_df = resampled_FB_data_df.drop('天格_Structure',1)
resampled_FB_data_df = resampled_FB_data_df.drop('人格_Structure',1)
resampled_FB_data_df = resampled_FB_data_df.drop('地格_Structure',1)
resampled_FB_data_df = resampled_FB_data_df.drop('外格_Structure',1)
resampled_FB_data_df = resampled_FB_data_df.drop('總格_Structure',1)
resampled_FB_data_df = resampled_FB_data_df.drop('三才_State',1)

In [41]:
resampled_FB_data_df['FN1_muin'] = resampled_FB_data_df['FN1_mu_in']
resampled_FB_data_df['FN2_muin'] = resampled_FB_data_df['FN2_mu_in']
resampled_FB_data_df = resampled_FB_data_df.drop('FN1_mu_in',1)
resampled_FB_data_df = resampled_FB_data_df.drop('FN2_mu_in',1)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

def dataset_statistics(dataset):
    """
    Basic statistics of the dataset
    :param dataset: Pandas dataframe
    :return: None, print the basic statistics of the dataset
    """
    print (dataset.describe())
    
#http://dataaspirant.com/2017/06/26/random-forest-classifier-python-scikit-learn/
#Split train and test
def split_dataset(dataset, train_percentage, feature_headers, target_header):
    """
    Split the dataset with train_percentage
    :param dataset:
    :param train_percentage:
    :param feature_headers:
    :param target_header:
    :return: train_x, test_x, train_y, test_y
    """

    # Split dataset into train and test dataset
    train_x, test_x, train_y, test_y = train_test_split(dataset[feature_headers], dataset[target_header],
                                                        train_size = train_percentage)
    return train_x, test_x, train_y, test_y

def random_forest_classifier(features, target):
    """
    To train the random forest classifier with features and target data
    :param features:
    :param target:
    :return: trained random forest classifier
    """
    clf = RandomForestClassifier(n_estimators = 33, n_jobs = -1 , min_samples_leaf = 1)
    clf.fit(features, target)
    return clf

In [13]:
#印出猜錯的年齡和實際年齡差距,算絕對值的差
#印出Test data的INDEX
#print(test_x.iloc[0].name)
#iloc[0].name是印第一列的編號名字
#loc[123].name會印編號名字123的編號名字，嗯，就是123
#用Total_df.loc[index]可以印該編號名字的那列....好像有重複的編號名字

def calculate_error(predictions,test_y):
    Error = 0
    list_y = list(test_y)
    
    for i in range(0, len(predictions)):
        if(list_y[i] != predictions[i]):
            #index = test_x.iloc[i].name
            #print('Name:', Totalname_list[int(test_x.iloc[i]['FN1'])], Totalname_list[int(test_x.iloc[i]['FN2'])])
            #print('BirthYear:', list_y[i], str(list_y[i]*5+1940))
            #print('Predicted outcome:', predictions[i], str(predictions[i]*5+1940))

            Error += abs(list_y[i] - predictions[i])
            
    print('平均絕對值誤差：', Error / len(predictions))
    print('RMSE:', sqrt(mean_squared_error(list_y,predictions)))

def validate_error(predictions,test_y,test_x,data_df):
    validation_list = []
    list_y = list(test_y)
    for i in range(10):
        validation_list.append(random.randint(0, len(predictions)))
        
    for i in range(0, len(predictions)):
            if i in validation_list:
                df_index = test_x.iloc[i].name
                name = data_df.loc[df_index].FirstName
                
                #print("Name:", Totalname_list[int (test_x.iloc[i].FN1)],Totalname_list[int(test_x.iloc[i].FN2)])
                print('Name', name)
                if 'GuessedGender' in test_x.columns.values:
                    print('Gender:',test_x.iloc[i].GuessedGender)
                print('answer:',list_y[i],'Classifier result:', predictions[i])
                
def calculate_error_multi_answer(predictions,test_x,name_dict_list,data_df):
    Error = 0
    Min_error_answer_list = []
    
    #print(test_x.iloc[0].name)
    for index,year in enumerate(predictions):
        df_index = test_x.iloc[index].name
        name = data_df.loc[df_index].FirstName
        
        #name = Totalname_list[int(test_x.iloc[index].FN1)] + Totalname_list[int(test_x.iloc[index].FN2)]
        
        if name in name_dict_list:
            if year not in name_dict_list[name]:
                smallest_dist = 99
                min_error_answer = 0
                for answer in (name_dict_list[name]):
                    dist = abs(answer - predictions[index])
                    if(dist < smallest_dist):
                        smallest_dist = dist
                        min_error_answer = answer

                Error += smallest_dist
                Min_error_answer_list.append(min_error_answer)
            else:
                Min_error_answer_list.append(year)
            
        else:
            print('OOPS')
            print('1', df_index)
            print('2', name,test_x.iloc[index])
            print('3', data_df.loc[df_index])

    print('平均絕對值誤差：', round(Error / len(predictions), 4))
    print('RMSE:', round(sqrt(mean_squared_error(Min_error_answer_list,predictions)), 4))
    
def validate_error_multi_answer(predictions, data_df, test_y, test_x, name_dict_list):
    validation_list = []
    list_y = list(test_y)
    for i in range(10):
        validation_list.append(random.randint(0, len(predictions)))
        
    for index,year in enumerate(predictions):
            if index in validation_list:
                df_index =  test_x.iloc[index].name
                name = data_df.loc[df_index].FirstName

                if name in name_dict_list:
                    print("Name:", name)
                    if 'GuessedGender' in test_x.columns.values:
                        print('Gender:', test_x.iloc[i].GuessedGender)
                    print('answer:', name_dict_list[name], 'Classifier result:', predictions[index])
                    

In [134]:
reconcatedata = pd.concat([resampled_Real_data_df, resample_taiwandata], sort = False)

In [151]:
reconcatedata = reconcatedata.drop('C_Name',1)
reconcatedata = reconcatedata.drop('Age',1)

In [152]:
# SAVE TAIWAN DATA INTO CSV FILE
reconcatedata.to_csv('./NameData/EnglishReconcateRealTaiwan.csv', index = False)

In [153]:
# READ RECONCATE DATA
reconcatedata = pd.read_csv('./NameData/EnglishReconcateRealTaiwan.csv', parse_dates = True)

/home/yvonnechow/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning:

Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.



In [156]:
reconcatedata.head()

,FirstName,LastName,gender,message,name,FN1,FN2,FN1_sonin,FN2_sonin,FN1_sonin_index,...,FN1_muin_index,FN2_muin_index,BirthYear,GuessedGender,BFN12P,FirstName1,FirstName2,FN1_muin,FN2_muin,Gender
0,秀珠,吳,NaN,1947,吳秀珠,70,596,x,zh,16,...,62,160,1,0,1.0,秀,珠,iu,u,NaN
1,豐博,林,NaN,1945,林豐博,277,72,f,b,13,...,25,154,1,1,6.0,豐,博,eng,o,NaN
2,義勝,康,1.0,1945,康義勝,650,156,y,sh,3,...,164,25,1,1,1.0,義,勝,i,eng,NaN
3,麗,吳,0.0,1946,吳麗,8578,203,-1,l,-1,...,230,164,1,0,6.0,,麗,-1,i,NaN
4,淡浪,張,NaN,1948,張淡浪,1008,104,d,l,1,...,80,27,1,1,6.0,淡,浪,an,ang,NaN


In [70]:
def basic_RFClassifier(data_df,x_feature,y_feature,Validation_times):

    #N fold cross-validation
    Train_accuracy_score = 0
    Test_accuracy_score = 0
    
    
    for i in range(Validation_times):
        train_x, test_x, train_y, test_y = split_dataset(data_df, 0.7, x_feature, y_feature)
 
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y)
        
        test_x = pd.concat([test_x, resample_taiwandata])
        test_y = pd.concat([test_y, resample_taiwandata])
        predictions = trained_model.predict(test_x[x_feature])
#         print(predictions)

        # Train and Test Accuracy
        if(Validation_times!=1):
            print('Run times:', i+1)
        print ("Train Accuracy :: ", accuracy_score(train_y, trained_model.predict(train_x)))
        print ("Test Accuracy  :: ", accuracy_score(test_y[y_feature], predictions))
        #print ("Confusion matrix\n ", confusion_matrix(test_y, predictions))

        Train_accuracy_score += accuracy_score(train_y, trained_model.predict(train_x))
        Test_accuracy_score += accuracy_score(test_y[y_feature], predictions)
        #calculate_error(predictions, test_y )
        #validate_error(predictions, test_y, test_x , data_df)
        
    if(Validation_times != 1):
        Train_accuracy_score /= Validation_times
        Test_accuracy_score /= Validation_times
        print ("Average Train Accuracy :: ", Train_accuracy_score)
        print ("Average Test Accuracy  :: ", Test_accuracy_score)
    
    return trained_model

In [115]:
# 6F, maintain previous muin index

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.5570493040602038
Test Accuracy  ::  0.2


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.5565653650631562
Test Accuracy  ::  0.20286957422562935


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.5561125047172952
Test Accuracy  ::  0.20049725473945923


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.5565076475680956
Test Accuracy  ::  0.20306640422666528


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.5565032077607832
Test Accuracy  ::  0.20124313684864809
Average Train Accuracy ::  0.556213732324017
Average Test Accuracy  ::  0.20167616285092715


In [74]:
def basic_RFClassifier(data_df,x_feature,y_feature,Validation_times):

    #N fold cross-validation
    Train_accuracy_score = 0
    Test_accuracy_score = 0
    
    for i in range(Validation_times):
        train_x, test_x, train_y, test_y = split_dataset(data_df, 0.7, x_feature, y_feature)
 
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y)
        
#         test_x = pd.concat([test_x, resample_taiwandata])
#         test_y = pd.concat([test_y, resample_taiwandata])
        predictions = trained_model.predict(test_x)
#         print(predictions)

        # Train and Test Accuracy
        if(Validation_times!=1):
            print('Run times:', i+1)
        print ("Train Accuracy :: ", accuracy_score(train_y, trained_model.predict(train_x)))
        print ("Test Accuracy  :: ", accuracy_score(test_y, predictions))
        #print ("Confusion matrix\n ", confusion_matrix(test_y, predictions))

        Train_accuracy_score += accuracy_score(train_y, trained_model.predict(train_x))
        Test_accuracy_score += accuracy_score(test_y, predictions)
        #calculate_error(predictions, test_y )
        #validate_error(predictions, test_y, test_x , data_df)
        
    if(Validation_times != 1):
        Train_accuracy_score /= Validation_times
        Test_accuracy_score /= Validation_times
        print ("Average Train Accuracy :: ", Train_accuracy_score)
        print ("Average Test Accuracy  :: ", Test_accuracy_score)
    
    return trained_model

# Gender

In [75]:
# 6F, maintain previous muin index

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
y_feature = ['gender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.9622948898265354
Test Accuracy  ::  0.8614833451840508


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.9625761837787155
Test Accuracy  ::  0.8630148225127168


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.9625644631973745
Test Accuracy  ::  0.8630968659410381


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.9619315518049695
Test Accuracy  ::  0.8635344308920856


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.9623886544772621
Test Accuracy  ::  0.8606629109008368
Average Train Accuracy ::  0.9623183309892169
Average Test Accuracy  ::  0.8618908275447138


In [103]:
#Read uni-gram classifier

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
    
result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

In [104]:
gender_list = resample_taiwandata['GuessedGender'].tolist()

In [105]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == result[x]:
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

553
985
0.5614213197969543


# get_test_name_dict_list

In [93]:
import datetime
import random
def get_test_name_dict_list(test_index,data_df,year_column):
    test_name_dict_list = {}
    FN_list = data_df.loc[test_index]['FirstName'].tolist()
    #year - message / birthyear
    Year_list = data_df.loc[test_index][year_column].tolist()
    for index in range(len(FN_list)):
        FN = FN_list[index]
        year = Year_list[index]
        #print(year)
        if FN in test_name_dict_list:
            if year not in test_name_dict_list[FN]:
                test_name_dict_list[FN][year] = 1
            else:
                test_name_dict_list[FN][year] += 1
        else:
            test_name_dict_list[FN] = {}
            test_name_dict_list[FN][year] = 1
            
    return test_name_dict_list

def get_min_distance(a,b_list):
    smallest_dist = 999
    a = 1940 + a * 5
    
    min_error_answer = 0
    for number in b_list:
        if int(number) > a:
            year = a + 4
        else:
            year = a
        dist = abs(year - int(number))
        if(dist < smallest_dist):
            smallest_dist = dist
    #print(a,b_list,smallest_dist)
    return smallest_dist

In [62]:
test_name_dict_list = get_test_name_dict_list(test_index,reconcatedata,'BirthYear')
test_name_year_dict_list = get_test_name_dict_list(test_index,reconcatedata,'message')

# print(test_name_dict_list)
# print(test_name_year_dict_list)

new_name_list = []
always_name_list = []
old_name_list = []
#1~5是old 6~10是new 兩者都有是always
for name in test_name_dict_list:
    new_flag = False
    old_flag = False
    for year in test_name_dict_list[name]:
        year = int(year)
        if year >= 1 and year <= 5:
            old_flag = True
        if year >= 6 and year <= 10:
            new_flag = True
    if new_flag and old_flag:
        always_name_list.append(name)
    elif new_flag:
        new_name_list.append(name)
    elif old_flag:
        old_name_list.append(name)
    else:
        print(name,'???')
        
print('len new_name_list',len(new_name_list),'len old_name_list',len(old_name_list))
print('len always_name_list',len(always_name_list))


Yi Ching ???
Yu Zhen ???
Pin Han ???
Yi Xuan ???
 Chieh ???
Yi Tang ???
Chia Chun ???
Ying Shin ???
Hsin Tien ???
Ying Yen ???
Tzu Yi ???
Pei Wun ???
Chieh Hsin ???
Yan Jun ???
Yu Cheng ???
Jhih Yun ???
Yun Ju ???
Shao Chuan ???
Yi Chuan ???
Ting Yu ???
Ting Yao ???
Chia Ying ???
Yu Jen ???
Hao Yun ???
Ching Yuan ???
Hong Wei ???
Jie Ming ???
Yi Yun ???
Sih Yun ???
Xin Yu ???
Yi Ting ???
Shuo YUn ???
Zi Qing ???
Li Hua ???
Su Yu ???
Chi Ting ???
Chun Wei ???
 Yang ???
Shi Ying ???
Chih Chun ???
En Tzu ???
Ko Chi ???
En Shou ???
Hsuan Yu ???
Yu Hsin ???
 Lin ???
Cai Jie ???
Tai Sheng ???
Yi Hsuan ???
Yu Pei ???
Chun Han ???
I Chen ???
Kuan Hung ???
Chih Kai ???
Pei Shan ???
Jo Hsuan ???
Chia Hao ???
Po Hsuan ???
Shiau Cheng ???
Yu Chi ???
Wei Hsu ???
Yu Ching ???
len new_name_list 22006 len old_name_list 21042
len always_name_list 5981


In [166]:
y_feature ='BirthYear'
train_x, test_x, train_y, test_y = split_dataset(reconcatedata, 0.7, x_feature, y_feature)
train_index = train_x.index
test_index = test_x.index
print(len(test_index))

%xdel train_x
%xdel test_x
%xdel train_y
%xdel test_y

96530


In [167]:
# SAVE TRAIN TEST INDEX
filename = 'Englishtest_index'
pickle.dump(test_index,open(filename, 'wb'))

filename = 'Englishtrain_index'
pickle.dump(train_index,open(filename, 'wb'))

In [168]:
# READ TRAIN TEST INDEX
with open('Englishtrain_index', 'rb') as handle:
    train_index = pickle.loads(handle.read())
    
with open('Englishtest_index', 'rb') as handle:
    test_index = pickle.loads(handle.read())

In [44]:
# multi-answer RFC
def basic_RFClassifier_ts(data_df,x_feature,y_feature,Validation_times,mode,train_index,test_index,random_avg_year_error):
    single_year_number = 0
    print(datetime.datetime.now())
    #N fold cross-validation
    Train_accuracy_score = 0
    Test_accuracy_score = 0
    
    avg_dist_error = 0
    avg_random_score = 0
    avg_random_dist_error = 0
    avg_single_year_error = 0
    avg_year_error = [0 for x in range(10)]
    
    old_score = 0
    new_score = 0
    always_score = 0
    old_count = 0
    new_count = 0
    always_count = 0
    
    if mode != 2:
        random_avg_year_error = [0 for x in range(10)]
    for i in range(Validation_times):
        random_score = 0
        if mode == 0:
            train_x, test_x, train_y, test_y = split_dataset(data_df, 0.7, x_feature, y_feature)
            train_index = train_x.index
            test_index = test_x.index
        else:
            train_x = data_df.loc[train_index][x_feature]
            test_x = data_df.loc[test_index][x_feature]
            train_y = data_df.loc[train_index][y_feature]
            train_y = train_y.values.reshape(-1, 1)
            test_y = data_df.loc[test_index][y_feature]
            test_y = test_y.values.reshape(-1, 1)
        
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y.ravel())
        print('Finished training')
        
        predictions = trained_model.predict(test_x)
        print('Finished prediction')
        # Train and Test Accuracy
        if(Validation_times != 1):
            print('Run times:',i+1)
        
        test_name_dict_list = get_test_name_dict_list(test_index,data_df,'BirthYear')
        test_name_year_dict_list = get_test_name_dict_list(test_index,data_df,'message')
        print('test dataset 有',len(test_x),'個名字,',len(test_name_dict_list),'種名字')    
        
        test_name_list = data_df.loc[test_index].FirstName.tolist()
#         print(len(test_name_list))
        
        dist_error = 0
        accuracy = 0
        random_dist_error = 0
        single_year_error = 0
        year_error = [0 for x in range(10)]
        year_count = [0 for x in range(10)]
        
        if mode != 2:
            random_year_error = [0 for x in range(10)]
            
        for test_no,year in enumerate(predictions):
            
            #print(test_x.iloc[index])
            #name = Totalname_list[int(test_x.iloc[index].FN1)]+Totalname_list[int(test_x.iloc[index].FN2)]
            #print(name)
            
            ##test 是不是印正確的名字
#             df_index = test_x.iloc[index].name
#             name = data_df.loc[df_index].FirstName
            #print(name)
            name = test_name_list[test_no]
            if name in test_name_dict_list:
                year_count[len(test_name_dict_list[name])-1] += 1

                if year in test_name_dict_list[name]:
                    accuracy+=1
                else:
                    #計算錯誤，把平均絕對值誤差加上去
                    error = get_min_distance(year,test_name_year_dict_list[name]) 
                    dist_error += error
                    
                    year_error[len(test_name_dict_list[name])-1] += error
                    
                    if name in old_name_list:
                        old_score += error
                        old_count += 1
                    elif name in new_name_list:
                        new_score += error
                        new_count += 1

                    elif name in always_name_list:
                        always_score += error
                        always_count += 1
      
                if mode != 2:
                    random_year = random.randint(1,10)
                    if random_year in test_name_dict_list[name]:
                        random_score += 1
                    else:
                        error = get_min_distance(random_year,test_name_year_dict_list[name])
                        random_dist_error += error
                        random_year_error[ len(test_name_dict_list[name])-1] += error
            else:
                print('arghhh test name not in test_name_dict_list')
        
        accuracy/=len(predictions)      

        print ("Train Accuracy :: ", accuracy_score(train_y, trained_model.predict(train_x)))
        print ("Test Accuracy  :: ", accuracy)
        print('平均年份絕對值誤差::',dist_error/len(predictions))
        #print('只在某一年份出現的平均年份絕對值誤差::',single_year_error/single_year_number)
        print('old::',old_score/old_count)
        print('new::',new_score/new_count)
        print('always::',always_score/always_count)   
        
        for i in range(len(year_error)):
            avg_year_error[i] += year_error[i]/year_count[i]
            if mode != 2:
                random_avg_year_error[i] = random_year_error[i]/year_count[i]
                random_avg_year_error[i] += random_year_error[i]/year_count[i]
            print('名字只出現在共',i+1,'個年份區間的名字比例::',round(year_count[i]/len(predictions),4)*100,'%')
            print('名字只出現在共',i+1,'個年份區間的平均誤差::',round(year_error[i]/year_count[i],4))
            if mode != 2:
                print('random 名字只出現在共',i+1,'個年份區間的平均誤差::',round(random_year_error[i]/year_count[i],4))
        if mode == 1:    
            print('Random guess accuracy::',random_score/len(predictions))
            print('Random guess error::',random_dist_error/len(predictions))

        #print (" Confusion matrix\n ", confusion_matrix(test_y, predictions))
        
        Train_accuracy_score += accuracy_score(train_y, trained_model.predict(train_x))
        Test_accuracy_score += accuracy
        avg_dist_error += dist_error/len(predictions)
        avg_random_score += random_score/len(predictions)
        avg_random_dist_error = random_dist_error/len(predictions)
        
#         calculate_errorr_multi_answer(predictions, test_x ,test_name_dict_list,data_df)
#         validate_error_multi_answer(predictions, data_df,test_y,test_x ,test_name_dict_list)
        
    if(Validation_times != 1):
        Train_accuracy_score/=Validation_times
        Test_accuracy_score/=Validation_times
        print ("Average Train Accuracy :: ", Train_accuracy_score)
        print ("Average Test Accuracy  :: ", Test_accuracy_score)
        print('Average 平均年份絕對值誤差::', avg_dist_error/Validation_times )
        for i in range(len(year_error)):
            print('Average 出現在共',i+1,'年份區間的平均誤差::',round(avg_year_error[i]/Validation_times,4))
            print('random 名字只出現在共',i+1,'個年份區間的平均誤差::',round(random_avg_year_error[i]/Validation_times,4))
        if mode != 2:
            print('Average Random guess accuracy::',avg_random_score/Validation_times)
            print('Average Random guess error::',avg_random_dist_error/Validation_times)  
        
        for i in range(9):
            improve_rate = abs(avg_year_error[i]-random_avg_year_error[i])/random_avg_year_error[i]
            print('名字只出現在共',i+1,'個年份區間的improve rate:', round(improve_rate*100,2),'%')
    return random_avg_year_error

In [50]:
# 6F
x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']
    
# age_test = basic_RFClassifier_ts(resampled_Real_data_df, x_feature, y_feature, 10)
age_test = basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-19 15:18:19.878746
Finished training
Finished prediction
Run times: 1
test dataset 有 97097 個名字, 49091 種名字
Train Accuracy ::  0.5573072476813244
Test Accuracy  ::  0.510170242129005
平均年份絕對值誤差:: 5.000175082649309
old:: 11.662179034797242
new:: 11.628241627413427
always:: 4.64725117514817
名字只出現在共 1 個年份區間的名字比例:: 40.38 %
名字只出現在共 1 個年份區間的平均誤差:: 9.8989
random 名字只出現在共 1 個年份區間的平均誤差:: 15.0605
名字只出現在共 2 個年份區間的名字比例:: 15.340000000000002 %
名字只出現在共 2 個年份區間的平均誤差:: 4.29
random 名字只出現在共 2 個年份區間的平均誤差:: 9.6244
名字只出現在共 3 個年份區間的名字比例:: 9.99 %
名字只出現在共 3 個年份區間的平均誤差:: 2.1303
random 名字只出現在共 3 個年份區間的平均誤差:: 6.8114
名字只出現在共 4 個年份區間的名字比例:: 7.489999999999999 %
名字只出現在共 4 個年份區間的平均誤差:: 1.0454
random 名字只出現在共 4 個年份區間的平均誤差:: 5.1084
名字只出現在共 5 個年份區間的名字比例:: 6.32 %
名字只出現在共 5 個年份區間的平均誤差:: 0.5775
random 名字只出現在共 5 個年份區間的平均誤差:: 3.5411
名字只出現在共 6 個年份區間的名字比例:: 5.84 %
名字只出現在共 6 個年份區間的平均誤差:: 0.1858
random 名字只出現在共 6 個年份區間的平均誤差:: 2.6516
名字只出現在共 7 個年份區間的名字比例:: 5.1 %
名字只出現在共 7 個年份區間的平均誤差:: 0.0812
random 名字只出現在共 7 個年份區間的平均誤差:: 1.5738


In [51]:
# Sonin+muin
x_feature = ['FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']
    
# age_test = basic_RFClassifier_ts(resampled_Real_data_df, x_feature, y_feature, 10)
age_test = basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-19 15:23:55.936068
Finished training
Finished prediction
Run times: 1
test dataset 有 97097 個名字, 49091 種名字
Train Accuracy ::  0.4061984309780355
Test Accuracy  ::  0.4898503558297373
平均年份絕對值誤差:: 5.484474288598
old:: 12.644424979389942
new:: 11.993568317633951
always:: 4.811769253067929
名字只出現在共 1 個年份區間的名字比例:: 40.38 %
名字只出現在共 1 個年份區間的平均誤差:: 10.717
random 名字只出現在共 1 個年份區間的平均誤差:: 15.0391
名字只出現在共 2 個年份區間的名字比例:: 15.340000000000002 %
名字只出現在共 2 個年份區間的平均誤差:: 4.8886
random 名字只出現在共 2 個年份區間的平均誤差:: 9.7693
名字只出現在共 3 個年份區間的名字比例:: 9.99 %
名字只出現在共 3 個年份區間的平均誤差:: 2.4907
random 名字只出現在共 3 個年份區間的平均誤差:: 6.9445
名字只出現在共 4 個年份區間的名字比例:: 7.489999999999999 %
名字只出現在共 4 個年份區間的平均誤差:: 1.2133
random 名字只出現在共 4 個年份區間的平均誤差:: 5.3224
名字只出現在共 5 個年份區間的名字比例:: 6.32 %
名字只出現在共 5 個年份區間的平均誤差:: 0.5923
random 名字只出現在共 5 個年份區間的平均誤差:: 3.4967
名字只出現在共 6 個年份區間的名字比例:: 5.84 %
名字只出現在共 6 個年份區間的平均誤差:: 0.3154
random 名字只出現在共 6 個年份區間的平均誤差:: 2.6287
名字只出現在共 7 個年份區間的名字比例:: 5.1 %
名字只出現在共 7 個年份區間的平均誤差:: 0.17
random 名字只出現在共 7 個年份區間的平均誤差:: 1.5669
名

In [52]:
# FN+muin
x_feature = ['FN1','FN2','FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']
    
# age_test = basic_RFClassifier_ts(resampled_Real_data_df, x_feature, y_feature, 10)
age_test = basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-19 15:27:37.237472
Finished training
Finished prediction
Run times: 1
test dataset 有 97097 個名字, 49091 種名字
Train Accuracy ::  0.5572190012266257
Test Accuracy  ::  0.5080589513579205
平均年份絕對值誤差:: 5.058271625281934
old:: 11.62125311113516
new:: 11.847774450067128
always:: 4.686835980977436
名字只出現在共 1 個年份區間的名字比例:: 40.38 %
名字只出現在共 1 個年份區間的平均誤差:: 10.0484
random 名字只出現在共 1 個年份區間的平均誤差:: 15.1115
名字只出現在共 2 個年份區間的名字比例:: 15.340000000000002 %
名字只出現在共 2 個年份區間的平均誤差:: 4.2378
random 名字只出現在共 2 個年份區間的平均誤差:: 9.7283
名字只出現在共 3 個年份區間的名字比例:: 9.99 %
名字只出現在共 3 個年份區間的平均誤差:: 2.1509
random 名字只出現在共 3 個年份區間的平均誤差:: 6.951
名字只出現在共 4 個年份區間的名字比例:: 7.489999999999999 %
名字只出現在共 4 個年份區間的平均誤差:: 1.0721
random 名字只出現在共 4 個年份區間的平均誤差:: 5.1278
名字只出現在共 5 個年份區間的名字比例:: 6.32 %
名字只出現在共 5 個年份區間的平均誤差:: 0.576
random 名字只出現在共 5 個年份區間的平均誤差:: 3.4304
名字只出現在共 6 個年份區間的名字比例:: 5.84 %
名字只出現在共 6 個年份區間的平均誤差:: 0.2294
random 名字只出現在共 6 個年份區間的平均誤差:: 2.6839
名字只出現在共 7 個年份區間的名字比例:: 5.1 %
名字只出現在共 7 個年份區間的平均誤差:: 0.0689
random 名字只出現在共 7 個年份區間的平均誤差:: 1.564

In [53]:
# FN+sonin
x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index']
y_feature = ['BirthYear']
    
# age_test = basic_RFClassifier_ts(resampled_Real_data_df, x_feature, y_feature, 10)
age_test = basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-19 15:31:20.703669
Finished training
Finished prediction
Run times: 1
test dataset 有 97097 個名字, 49091 種名字
Train Accuracy ::  0.5571704656765415
Test Accuracy  ::  0.5063802177204239
平均年份絕對值誤差:: 5.120178790281883
old:: 11.822593691021838
new:: 11.893818927930239
always:: 4.6222605519480515
名字只出現在共 1 個年份區間的名字比例:: 40.38 %
名字只出現在共 1 個年份區間的平均誤差:: 10.154
random 名字只出現在共 1 個年份區間的平均誤差:: 15.0333
名字只出現在共 2 個年份區間的名字比例:: 15.340000000000002 %
名字只出現在共 2 個年份區間的平均誤差:: 4.349
random 名字只出現在共 2 個年份區間的平均誤差:: 9.7669
名字只出現在共 3 個年份區間的名字比例:: 9.99 %
名字只出現在共 3 個年份區間的平均誤差:: 2.1827
random 名字只出現在共 3 個年份區間的平均誤差:: 6.9324
名字只出現在共 4 個年份區間的名字比例:: 7.489999999999999 %
名字只出現在共 4 個年份區間的平均誤差:: 1.103
random 名字只出現在共 4 個年份區間的平均誤差:: 5.3057
名字只出現在共 5 個年份區間的名字比例:: 6.32 %
名字只出現在共 5 個年份區間的平均誤差:: 0.5411
random 名字只出現在共 5 個年份區間的平均誤差:: 3.4463
名字只出現在共 6 個年份區間的名字比例:: 5.84 %
名字只出現在共 6 個年份區間的平均誤差:: 0.1846
random 名字只出現在共 6 個年份區間的平均誤差:: 2.589
名字只出現在共 7 個年份區間的名字比例:: 5.1 %
名字只出現在共 7 個年份區間的平均誤差:: 0.0887
random 名字只出現在共 7 個年份區間的平均誤差:: 1.516

In [54]:
# Muin
x_feature = ['FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']
    
# age_test = basic_RFClassifier_ts(resampled_Real_data_df, x_feature, y_feature, 10)
age_test = basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-19 15:34:53.915053
Finished training
Finished prediction
Run times: 1
test dataset 有 97097 個名字, 49091 種名字
Train Accuracy ::  0.22746406163132396
Test Accuracy  ::  0.4403328630132754
平均年份絕對值誤差:: 6.948896464360382
old:: 15.034407582938389
new:: 13.53200075075075
always:: 5.716944584807142
名字只出現在共 1 個年份區間的名字比例:: 40.38 %
名字只出現在共 1 個年份區間的平均誤差:: 12.9624
random 名字只出現在共 1 個年份區間的平均誤差:: 15.0642
名字只出現在共 2 個年份區間的名字比例:: 15.340000000000002 %
名字只出現在共 2 個年份區間的平均誤差:: 6.5857
random 名字只出現在共 2 個年份區間的平均誤差:: 9.6461
名字只出現在共 3 個年份區間的名字比例:: 9.99 %
名字只出現在共 3 個年份區間的平均誤差:: 3.9421
random 名字只出現在共 3 個年份區間的平均誤差:: 6.9854
名字只出現在共 4 個年份區間的名字比例:: 7.489999999999999 %
名字只出現在共 4 個年份區間的平均誤差:: 2.2609
random 名字只出現在共 4 個年份區間的平均誤差:: 5.2912
名字只出現在共 5 個年份區間的名字比例:: 6.32 %
名字只出現在共 5 個年份區間的平均誤差:: 1.1595
random 名字只出現在共 5 個年份區間的平均誤差:: 3.4665
名字只出現在共 6 個年份區間的名字比例:: 5.84 %
名字只出現在共 6 個年份區間的平均誤差:: 0.6987
random 名字只出現在共 6 個年份區間的平均誤差:: 2.6936
名字只出現在共 7 個年份區間的名字比例:: 5.1 %
名字只出現在共 7 個年份區間的平均誤差:: 0.3963
random 名字只出現在共 7 個年份區間的平均誤差:: 1.

In [55]:
# Sonin
x_feature = ['FN1_sonin_index','FN2_sonin_index']
y_feature = ['BirthYear']
    
# age_test = basic_RFClassifier_ts(resampled_Real_data_df, x_feature, y_feature, 10)
age_test = basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-19 15:39:05.985072
Finished training
Finished prediction
Run times: 1
test dataset 有 97097 個名字, 49091 種名字
Train Accuracy ::  0.16560329688754755
Test Accuracy  ::  0.4094462238792136
平均年份絕對值誤差:: 7.565218286867771
old:: 16.2285210152468
new:: 13.46802900461437
always:: 6.359936840594273
名字只出現在共 1 個年份區間的名字比例:: 40.38 %
名字只出現在共 1 個年份區間的平均誤差:: 13.5201
random 名字只出現在共 1 個年份區間的平均誤差:: 15.112
名字只出現在共 2 個年份區間的名字比例:: 15.340000000000002 %
名字只出現在共 2 個年份區間的平均誤差:: 7.225
random 名字只出現在共 2 個年份區間的平均誤差:: 9.7014
名字只出現在共 3 個年份區間的名字比例:: 9.99 %
名字只出現在共 3 個年份區間的平均誤差:: 4.7616
random 名字只出現在共 3 個年份區間的平均誤差:: 7.0312
名字只出現在共 4 個年份區間的名字比例:: 7.489999999999999 %
名字只出現在共 4 個年份區間的平均誤差:: 2.7527
random 名字只出現在共 4 個年份區間的平均誤差:: 5.2962
名字只出現在共 5 個年份區間的名字比例:: 6.32 %
名字只出現在共 5 個年份區間的平均誤差:: 1.9576
random 名字只出現在共 5 個年份區間的平均誤差:: 3.4784
名字只出現在共 6 個年份區間的名字比例:: 5.84 %
名字只出現在共 6 個年份區間的平均誤差:: 2.0889
random 名字只出現在共 6 個年份區間的平均誤差:: 2.6768
名字只出現在共 7 個年份區間的名字比例:: 5.1 %
名字只出現在共 7 個年份區間的平均誤差:: 1.0333
random 名字只出現在共 7 個年份區間的平均誤差:: 1.52
名

In [56]:
# FN
x_feature = ['FN1','FN2']
y_feature = ['BirthYear']
    
# age_test = basic_RFClassifier_ts(resampled_Real_data_df, x_feature, y_feature, 10)
age_test = basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-19 15:44:09.686025
Finished training
Finished prediction
Run times: 1
test dataset 有 97097 個名字, 49091 種名字
Train Accuracy ::  0.5570248590262886
Test Accuracy  ::  0.5047220820416697
平均年份絕對值誤差:: 5.218760620822477
old:: 11.974127265881341
new:: 12.158784649280115
always:: 4.655103066867773
名字只出現在共 1 個年份區間的名字比例:: 40.38 %
名字只出現在共 1 個年份區間的平均誤差:: 10.3562
random 名字只出現在共 1 個年份區間的平均誤差:: 15.0715
名字只出現在共 2 個年份區間的名字比例:: 15.340000000000002 %
名字只出現在共 2 個年份區間的平均誤差:: 4.4572
random 名字只出現在共 2 個年份區間的平均誤差:: 9.6567
名字只出現在共 3 個年份區間的名字比例:: 9.99 %
名字只出現在共 3 個年份區間的平均誤差:: 2.1983
random 名字只出現在共 3 個年份區間的平均誤差:: 6.9376
名字只出現在共 4 個年份區間的名字比例:: 7.489999999999999 %
名字只出現在共 4 個年份區間的平均誤差:: 1.0565
random 名字只出現在共 4 個年份區間的平均誤差:: 5.2693
名字只出現在共 5 個年份區間的名字比例:: 6.32 %
名字只出現在共 5 個年份區間的平均誤差:: 0.5732
random 名字只出現在共 5 個年份區間的平均誤差:: 3.5457
名字只出現在共 6 個年份區間的名字比例:: 5.84 %
名字只出現在共 6 個年份區間的平均誤差:: 0.1948
random 名字只出現在共 6 個年份區間的平均誤差:: 2.6698
名字只出現在共 7 個年份區間的名字比例:: 5.1 %
名字只出現在共 7 個年份區間的平均誤差:: 0.0859
random 名字只出現在共 7 個年份區間的平均誤差:: 1.

# Word2Vec

In [157]:
#使用word to vector
from gensim.models import word2vec
# moe_model = word2vec.Word2Vec.load("./data_output/dict_to_vector/moe_med250.model.bin")
moe_model = word2vec.Word2Vec.load("./data_output/graduate/files/wiki_moe_100_model.bin")

In [158]:
%%time 

def add_word_vector(word,n):
    if word in moe_model.wv:
        return moe_model.wv[word][n]
    else:
        return 0
    
w_feature = []
x_feature = []
FirstName1_list = reconcatedata.FirstName1.tolist()
FirstName2_list = reconcatedata.FirstName2.tolist()

for i in range(0,100):
    tmp1_list = []
    tmp2_list = []
    for word in FirstName1_list:
        tmp1_list.append(add_word_vector(word,i))
    for word in FirstName2_list:
        tmp2_list.append(add_word_vector(word,i))

    reconcatedata['FN1_wv_'+str(i)] = tmp1_list
    reconcatedata['FN2_wv_'+str(i)] = tmp2_list
    
    x_feature.append('FN1_wv_'+str(i))
    x_feature.append('FN2_wv_'+str(i))
    w_feature.append('FN1_wv_'+str(i))
    w_feature.append('FN2_wv_'+str(i))

%xdel tmp1_list
%xdel tmp2_list


CPU times: user 2min 12s, sys: 464 ms, total: 2min 12s
Wall time: 2min 12s


# One Hot Encoding

In [159]:
def one_hot_encoding(df):
    df = pd.get_dummies(df, columns = ["FN1_sonin"])
    df = pd.get_dummies(df, columns = ["FN2_sonin"])
    df = pd.get_dummies(df, columns = ["FN1_muin"])
    df = pd.get_dummies(df, columns = ["FN2_muin"])
    df = pd.get_dummies(df, columns = ["FirstName2"])
    df = pd.get_dummies(df, columns = ["FirstName1"])
    df = pd.get_dummies(df, columns = ["LastName"])

    return df

In [160]:
reconcatedata = one_hot_encoding(reconcatedata)

In [161]:
reconcatedata.head()

,FirstName,gender,message,name,FN1,FN2,FN1_sonin_index,FN2_sonin_index,FN1_muin_index,FN2_muin_index,...,LastName_黃沈,LastName_黄,LastName_黎,LastName_黑,LastName_黨,LastName_齊,LastName_龍,LastName_龎,LastName_龐,LastName_龔
0,秀珠,NaN,1947,吳秀珠,70,596,16,8,62,160,...,0,0,0,0,0,0,0,0,0,0
1,豐博,NaN,1945,林豐博,277,72,13,7,25,154,...,0,0,0,0,0,0,0,0,0,0
2,義勝,1.0,1945,康義勝,650,156,3,5,164,25,...,0,0,0,0,0,0,0,0,0,0
3,麗,0.0,1946,吳麗,8578,203,-1,18,230,164,...,0,0,0,0,0,0,0,0,0,0
4,淡浪,NaN,1948,張淡浪,1008,104,1,18,80,27,...,0,0,0,0,0,0,0,0,0,0


In [162]:
character_feature = []
for index, x in enumerate(reconcatedata.columns.values):
    if 'FirstName' in x and '_' in x:
        character_feature.append(x)
print(len(character_feature))

7557


In [163]:
in_feature = []
for index, x in enumerate(reconcatedata.columns.values):
    if 'FN1_muin_' in x or 'FN2_muin_' in x or 'FN1_sonin_' in x or 'FN2_sonin_' in x:
        in_feature.append(x)
print(len(in_feature))

310


In [164]:
# Taiwan Dataset!!!
#每種feature 都是由很多欄位的value組成

def get_x_feature(feature_list):
    f = []
    
    if 'w2v' in feature_list:
        for x in w_feature:
            f.append(x)
    if 'character' in feature_list:
        for x in character_feature:
            f.append(x)
    if 'radical' in feature_list:
        for x in radial_feature:
            f.append(x)
    if 'in' in feature_list:
        for x in in_feature:
            f.append(x)
    if 'muin' in feature_list:
        for x in in_feature:
            f.append(x)
    if 'sonin' in feature_list:
        for x in in_feature:
            f.append(x)
#     if 'stroke' in feature_list:
#         for index, x in enumerate(reconcatedata.columns.values):
#             if '天格_' in x and  '天格_' in feature_list:
#                 f.append(x)
#             if '地格_' in x  and  '地格_' in feature_list:
#                 f.append(x)
#             if '人格_' in x  and  '人格_' in feature_list :
#                 f.append(x)
#             if '外格_' in x  and  '外格_' in feature_list:
#                 f.append(x)
#             if '總格_' in x  and  '總格_' in feature_list:
#                 f.append(x)
#             if '三才_' in x  and  '三才_' in feature_list:
#                 f.append(x)
    
    return f 
    
character_feature = []
for index, x in enumerate(reconcatedata.columns.values):
    if 'FirstName' in x and '_' in x:
        character_feature.append(x)
print('len of character_feature:',len(character_feature))

in_feature = []
for index, x in enumerate(reconcatedata.columns.values):
    if 'FN1_muin_' in x or 'FN2_muin_' in x or 'FN1_sonin_' in x or 'FN2_sonin_' in x:
        in_feature.append(x)
print('len of in_feature:',len(in_feature))

muin_feature = []
for index, x in enumerate(reconcatedata.columns.values):
    if 'FN1_muin_' in x or 'FN2_muin_' in x:
        muin_feature.append(x)
print('len of in_feature:',len(muin_feature))

sonin_feature = []
for index, x in enumerate(reconcatedata.columns.values):
    if 'FN1_sonin_' in x or 'FN2_sonin_' in x:
        sonin_feature.append(x)
print('len of in_feature:',len(sonin_feature))

LN_feature = []
for f in reconcatedata.columns.values:
    if 'LastName_' in f:
        LN_feature.append(f)
print('len of LN_feature:',len(LN_feature))

len of character_feature: 7557
len of in_feature: 310
len of in_feature: 162
len of in_feature: 148
len of LN_feature: 967


In [178]:
%%time
# Only character
x_f = ['character']

y_feature = 'BirthYear'
x_feature = get_x_feature(x_f)
# print(len(x_feature))

basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-21 19:50:38.052007
Finished training
Finished prediction
Run times: 1
test dataset 有 96530 個名字, 49071 種名字
Train Accuracy ::  0.5553355384376318
Test Accuracy  ::  0.5074070237231949
平均年份絕對值誤差:: 4.800093235263649
old:: 9.42504003202562
new:: 8.910274825241789
always:: 6.356902356902357
名字只出現在共 1 個年份區間的名字比例:: 40.48 %
名字只出現在共 1 個年份區間的平均誤差:: 9.4574
random 名字只出現在共 1 個年份區間的平均誤差:: 14.9517
名字只出現在共 2 個年份區間的名字比例:: 15.229999999999999 %
名字只出現在共 2 個年份區間的平均誤差:: 4.1321
random 名字只出現在共 2 個年份區間的平均誤差:: 9.6239
名字只出現在共 3 個年份區間的名字比例:: 9.83 %
名字只出現在共 3 個年份區間的平均誤差:: 2.0694
random 名字只出現在共 3 個年份區間的平均誤差:: 6.9227
名字只出現在共 4 個年份區間的名字比例:: 7.99 %
名字只出現在共 4 個年份區間的平均誤差:: 1.0306
random 名字只出現在共 4 個年份區間的平均誤差:: 5.2473
名字只出現在共 5 個年份區間的名字比例:: 6.12 %
名字只出現在共 5 個年份區間的平均誤差:: 0.5178
random 名字只出現在共 5 個年份區間的平均誤差:: 3.5352
名字只出現在共 6 個年份區間的名字比例:: 6.4399999999999995 %
名字只出現在共 6 個年份區間的平均誤差:: 0.2798
random 名字只出現在共 6 個年份區間的平均誤差:: 2.5417
名字只出現在共 7 個年份區間的名字比例:: 5.27 %
名字只出現在共 7 個年份區間的平均誤差:: 0.0868
random 名字只出現在共 7 個年份區間的平均誤差:: 1.53

[30.001688843398156,
 19.300693594451243,
 13.72699483503742,
 10.270382372002592,
 7.394041976980366,
 5.2674063354237015,
 3.079283887468031,
 1.2956343392909464,
 0.4857462269424259,
 0.0]

In [177]:
%%time
# Only muin
x_f = ['muin']

y_feature = 'BirthYear'
x_feature = get_x_feature(x_f)
# print(len(x_feature))

basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-21 19:44:13.817656
Finished training
Finished prediction
Run times: 1
test dataset 有 96530 個名字, 49071 種名字
Train Accuracy ::  0.3265034297511488
Test Accuracy  ::  0.47071376774059875
平均年份絕對值誤差:: 5.862768051382989
old:: 10.960545193687231
new:: 10.611235552369859
always:: 6.36422782787353
名字只出現在共 1 個年份區間的名字比例:: 40.48 %
名字只出現在共 1 個年份區間的平均誤差:: 11.382
random 名字只出現在共 1 個年份區間的平均誤差:: 15.0126
名字只出現在共 2 個年份區間的名字比例:: 15.229999999999999 %
名字只出現在共 2 個年份區間的平均誤差:: 5.2572
random 名字只出現在共 2 個年份區間的平均誤差:: 9.62
名字只出現在共 3 個年份區間的名字比例:: 9.83 %
名字只出現在共 3 個年份區間的平均誤差:: 2.6672
random 名字只出現在共 3 個年份區間的平均誤差:: 6.9684
名字只出現在共 4 個年份區間的名字比例:: 7.99 %
名字只出現在共 4 個年份區間的平均誤差:: 1.2722
random 名字只出現在共 4 個年份區間的平均誤差:: 5.0391
名字只出現在共 5 個年份區間的名字比例:: 6.12 %
名字只出現在共 5 個年份區間的平均誤差:: 0.6564
random 名字只出現在共 5 個年份區間的平均誤差:: 3.6307
名字只出現在共 6 個年份區間的名字比例:: 6.4399999999999995 %
名字只出現在共 6 個年份區間的平均誤差:: 0.5605
random 名字只出現在共 6 個年份區間的平均誤差:: 2.5745
名字只出現在共 7 個年份區間的名字比例:: 5.27 %
名字只出現在共 7 個年份區間的平均誤差:: 0.203
random 名字只出現在共 7 個年份區間的平均誤差:: 1.61

[30.011617195496417,
 19.240038079695363,
 13.826499420259303,
 10.374594944912507,
 7.166215301286392,
 5.141984241839524,
 3.0600039346842416,
 1.2921183709346615,
 0.45276690888764676,
 0.0]

In [176]:
%%time
# Only sonin
x_f = ['sonin']

y_feature = 'BirthYear'
x_feature = get_x_feature(x_f)
# print(len(x_feature))

basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-21 19:37:47.690063
Finished training
Finished prediction
Run times: 1
test dataset 有 96530 個名字, 49071 種名字
Train Accuracy ::  0.32648567052189936
Test Accuracy  ::  0.47085880037294103
平均年份絕對值誤差:: 5.873676577229877
old:: 10.960835275138914
new:: 10.627591778116866
always:: 6.493164730006836
名字只出現在共 1 個年份區間的名字比例:: 40.48 %
名字只出現在共 1 個年份區間的平均誤差:: 11.3831
random 名字只出現在共 1 個年份區間的平均誤差:: 15.0863
名字只出現在共 2 個年份區間的名字比例:: 15.229999999999999 %
名字只出現在共 2 個年份區間的平均誤差:: 5.2732
random 名字只出現在共 2 個年份區間的平均誤差:: 9.665
名字只出現在共 3 個年份區間的名字比例:: 9.83 %
名字只出現在共 3 個年份區間的平均誤差:: 2.7158
random 名字只出現在共 3 個年份區間的平均誤差:: 6.844
名字只出現在共 4 個年份區間的名字比例:: 7.99 %
名字只出現在共 4 個年份區間的平均誤差:: 1.2555
random 名字只出現在共 4 個年份區間的平均誤差:: 5.2538
名字只出現在共 5 個年份區間的名字比例:: 6.12 %
名字只出現在共 5 個年份區間的平均誤差:: 0.7146
random 名字只出現在共 5 個年份區間的平均誤差:: 3.5709
名字只出現在共 6 個年份區間的名字比例:: 6.4399999999999995 %
名字只出現在共 6 個年份區間的平均誤差:: 0.5853
random 名字只出現在共 6 個年份區間的平均誤差:: 2.6774
名字只出現在共 7 個年份區間的名字比例:: 5.27 %
名字只出現在共 7 個年份區間的平均誤差:: 0.1989
random 名字只出現在共 7 個年份區間的平均誤差:: 

[30.064994882292734,
 19.190262477900177,
 14.077790660904396,
 10.3800388852884,
 7.234935680433311,
 4.879562630648014,
 2.9620302970686603,
 1.2979783181951363,
 0.47344885410844045,
 0.0]

In [175]:
%%time
# Only muin+sonin
x_f = ['muin','sonin']

y_feature = 'BirthYear'
x_feature = get_x_feature(x_f)
# print(len(x_feature))

basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-21 19:31:11.778825
Finished training
Finished prediction
Run times: 1
test dataset 有 96530 個名字, 49071 種名字
Train Accuracy ::  0.3265433880169601
Test Accuracy  ::  0.47095203563658966
平均年份絕對值誤差:: 5.874826478814876
old:: 10.936958657022949
new:: 10.599910394265233
always:: 6.524954212454213
名字只出現在共 1 個年份區間的名字比例:: 40.48 %
名字只出現在共 1 個年份區間的平均誤差:: 11.3981
random 名字只出現在共 1 個年份區間的平均誤差:: 14.986
名字只出現在共 2 個年份區間的名字比例:: 15.229999999999999 %
名字只出現在共 2 個年份區間的平均誤差:: 5.2655
random 名字只出現在共 2 個年份區間的平均誤差:: 9.6369
名字只出現在共 3 個年份區間的名字比例:: 9.83 %
名字只出現在共 3 個年份區間的平均誤差:: 2.6827
random 名字只出現在共 3 個年份區間的平均誤差:: 6.9072
名字只出現在共 4 個年份區間的名字比例:: 7.99 %
名字只出現在共 4 個年份區間的平均誤差:: 1.317
random 名字只出現在共 4 個年份區間的平均誤差:: 5.2267
名字只出現在共 5 個年份區間的名字比例:: 6.12 %
名字只出現在共 5 個年份區間的平均誤差:: 0.6889
random 名字只出現在共 5 個年份區間的平均誤差:: 3.521
名字只出現在共 6 個年份區間的名字比例:: 6.4399999999999995 %
名字只出現在共 6 個年份區間的平均誤差:: 0.5549
random 名字只出現在共 6 個年份區間的平均誤差:: 2.6348
名字只出現在共 7 個年份區間的名字比例:: 5.27 %
名字只出現在共 7 個年份區間的平均誤差:: 0.171
random 名字只出現在共 7 個年份區間的平均誤差:: 1.6

[30.112026612077788,
 19.418332653338773,
 14.048276589016549,
 10.495139338950096,
 7.286729857819905,
 5.3741759125261295,
 3.207161125319693,
 1.2141810723703486,
 0.538848518725545,
 0.0]

In [174]:
%%time
# Only character,muin,sonin
x_f = ['character','muin','sonin']

y_feature = 'BirthYear'
x_feature = get_x_feature(x_f)
# print(len(x_feature))

basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-21 19:20:16.689147
Finished training
Finished prediction
Run times: 1
test dataset 有 96530 個名字, 49071 種名字
Train Accuracy ::  0.5552822607498834
Test Accuracy  ::  0.49929555578576607
平均年份絕對值誤差:: 5.037604889671605
old:: 9.659568145800318
new:: 9.07634016278417
always:: 6.297251277061542
名字只出現在共 1 個年份區間的名字比例:: 40.48 %
名字只出現在共 1 個年份區間的平均誤差:: 9.9949
random 名字只出現在共 1 個年份區間的平均誤差:: 15.0841
名字只出現在共 2 個年份區間的名字比例:: 15.229999999999999 %
名字只出現在共 2 個年份區間的平均誤差:: 4.308
random 名字只出現在共 2 個年份區間的平均誤差:: 9.7988
名字只出現在共 3 個年份區間的名字比例:: 9.83 %
名字只出現在共 3 個年份區間的平均誤差:: 2.0814
random 名字只出現在共 3 個年份區間的平均誤差:: 7.073
名字只出現在共 4 個年份區間的名字比例:: 7.99 %
名字只出現在共 4 個年份區間的平均誤差:: 1.011
random 名字只出現在共 4 個年份區間的平均誤差:: 5.129
名字只出現在共 5 個年份區間的名字比例:: 6.12 %
名字只出現在共 5 個年份區間的平均誤差:: 0.4661
random 名字只出現在共 5 個年份區間的平均誤差:: 3.6435
名字只出現在共 6 個年份區間的名字比例:: 6.4399999999999995 %
名字只出現在共 6 個年份區間的平均誤差:: 0.2203
random 名字只出現在共 6 個年份區間的平均誤差:: 2.542
名字只出現在共 7 個年份區間的名字比例:: 5.27 %
名字只出現在共 7 個年份區間的平均誤差:: 0.0995
random 名字只出現在共 7 個年份區間的平均誤差:: 1.5377
名

[30.041709314227226,
 19.397388820889432,
 14.094655844840307,
 10.470771224886585,
 7.234258632362898,
 5.1037144235407625,
 3.0596104662600827,
 1.2956343392909464,
 0.48462828395751817,
 0.0]

In [169]:
%%time
# Only w2v
x_f = ['w2v']

x_feature = get_x_feature(x_f)
y_feature ='BirthYear'

basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')


2019-11-21 18:43:30.599211
Finished training
Finished prediction
Run times: 1
test dataset 有 96530 個名字, 49071 種名字
Train Accuracy ::  0.5516993362488068
Test Accuracy  ::  0.5041023516005387
平均年份絕對值誤差:: 4.814140681653372
old:: 9.572960739938994
new:: 8.67674840187005
always:: 6.388202853374895
名字只出現在共 1 個年份區間的名字比例:: 40.48 %
名字只出現在共 1 個年份區間的平均誤差:: 9.4657
random 名字只出現在共 1 個年份區間的平均誤差:: 14.9498
名字只出現在共 2 個年份區間的名字比例:: 15.229999999999999 %
名字只出現在共 2 個年份區間的平均誤差:: 4.1704
random 名字只出現在共 2 個年份區間的平均誤差:: 9.7855
名字只出現在共 3 個年份區間的名字比例:: 9.83 %
名字只出現在共 3 個年份區間的平均誤差:: 2.1682
random 名字只出現在共 3 個年份區間的平均誤差:: 6.8693
名字只出現在共 4 個年份區間的名字比例:: 7.99 %
名字只出現在共 4 個年份區間的平均誤差:: 1.0333
random 名字只出現在共 4 個年份區間的平均誤差:: 5.2245
名字只出現在共 5 個年份區間的名字比例:: 6.12 %
名字只出現在共 5 個年份區間的平均誤差:: 0.4775
random 名字只出現在共 5 個年份區間的平均誤差:: 3.6491
名字只出現在共 6 個年份區間的名字比例:: 6.4399999999999995 %
名字只出現在共 6 個年份區間的平均誤差:: 0.2312
random 名字只出現在共 6 個年份區間的平均誤差:: 2.6146
名字只出現在共 7 個年份區間的名字比例:: 5.27 %
名字只出現在共 7 個年份區間的平均誤差:: 0.1009
random 名字只出現在共 7 個年份區間的平均誤差:: 1.49

[30.182497441146367,
 19.35577315381477,
 13.7113945398967,
 10.278677900194426,
 6.97291807718348,
 5.12301013024602,
 2.9250442651977178,
 1.2733665397011427,
 0.43991056456120736,
 0.0]

In [170]:
%%time
# w2v + character 
x_f = ['w2v','character']

y_feature = 'BirthYear'
x_feature = get_x_feature(x_f)
# print(len(x_feature))

basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-21 18:50:08.717867
Finished training
Finished prediction
Run times: 1
test dataset 有 96530 個名字, 49071 種名字
Train Accuracy ::  0.5553799365107555
Test Accuracy  ::  0.5037190510722055
平均年份絕對值誤差:: 4.878141510411271
old:: 9.610388330376503
new:: 8.894517958412099
always:: 6.352406352285126
名字只出現在共 1 個年份區間的名字比例:: 40.48 %
名字只出現在共 1 個年份區間的平均誤差:: 9.6178
random 名字只出現在共 1 個年份區間的平均誤差:: 15.0025
名字只出現在共 2 個年份區間的名字比例:: 15.229999999999999 %
名字只出現在共 2 個年份區間的平均誤差:: 4.2523
random 名字只出現在共 2 個年份區間的平均誤差:: 9.6727
名字只出現在共 3 個年份區間的名字比例:: 9.83 %
名字只出現在共 3 個年份區間的平均誤差:: 2.0553
random 名字只出現在共 3 個年份區間的平均誤差:: 6.8828
名字只出現在共 4 個年份區間的名字比例:: 7.99 %
名字只出現在共 4 個年份區間的平均誤差:: 1.0143
random 名字只出現在共 4 個年份區間的平均誤差:: 5.0913
名字只出現在共 5 個年份區間的名字比例:: 6.12 %
名字只出現在共 5 個年份區間的平均誤差:: 0.5396
random 名字只出現在共 5 個年份區間的平均誤差:: 3.6774
名字只出現在共 6 個年份區間的名字比例:: 6.4399999999999995 %
名字只出現在共 6 個年份區間的平均誤差:: 0.2201
random 名字只出現在共 6 個年份區間的平均誤差:: 2.5488
名字只出現在共 7 個年份區間的名字比例:: 5.27 %
名字只出現在共 7 個年份區間的平均誤差:: 0.0944
random 名字只出現在共 7 個年份區間的平均誤差:: 1.5

[30.14544524053224,
 19.284917720658235,
 14.208917466006113,
 10.5049902786779,
 7.47968855788761,
 5.074127673259366,
 3.124532756246311,
 1.2979783181951363,
 0.44494130799329235,
 0.0]

In [171]:
%%time
# w2v + muin&sonin 
x_f = ['w2v','in']

y_feature = 'BirthYear'
x_feature = get_x_feature(x_f)
# print(len(x_feature))

basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-21 19:01:05.638034
Finished training
Finished prediction
Run times: 1
test dataset 有 96530 個名字, 49071 種名字
Train Accuracy ::  0.5553754967034431
Test Accuracy  ::  0.5078006837252668
平均年份絕對值誤差:: 4.827607997513726
old:: 9.573726273726274
new:: 8.88671875
always:: 6.314258119038995
名字只出現在共 1 個年份區間的名字比例:: 40.48 %
名字只出現在共 1 個年份區間的平均誤差:: 9.5607
random 名字只出現在共 1 個年份區間的平均誤差:: 15.0033
名字只出現在共 2 個年份區間的名字比例:: 15.229999999999999 %
名字只出現在共 2 個年份區間的平均誤差:: 4.0301
random 名字只出現在共 2 個年份區間的平均誤差:: 9.6961
名字只出現在共 3 個年份區間的名字比例:: 9.83 %
名字只出現在共 3 個年份區間的平均誤差:: 2.098
random 名字只出現在共 3 個年份區間的平均誤差:: 6.9738
名字只出現在共 4 個年份區間的名字比例:: 7.99 %
名字只出現在共 4 個年份區間的平均誤差:: 1.0097
random 名字只出現在共 4 個年份區間的平均誤差:: 5.2805
名字只出現在共 5 個年份區間的名字比例:: 6.12 %
名字只出現在共 5 個年份區間的平均誤差:: 0.545
random 名字只出現在共 5 個年份區間的平均誤差:: 3.4824
名字只出現在共 6 個年份區間的名字比例:: 6.4399999999999995 %
名字只出現在共 6 個年份區間的平均誤差:: 0.2553
random 名字只出現在共 6 個年份區間的平均誤差:: 2.5794
名字只出現在共 7 個年份區間的名字比例:: 5.27 %
名字只出現在共 7 個年份區間的平均誤差:: 0.1007
random 名字只出現在共 7 個年份區間的平均誤差:: 1.556
名字只出現在

[30.134339815762537,
 19.260709914320685,
 13.978286075682513,
 10.346856772521063,
 7.1482735274204465,
 5.111111111111111,
 3.161125319693095,
 1.40111338997949,
 0.4589155953046395,
 0.0]

In [173]:
%%time
# w2v + muin 
x_f = ['w2v','muin']

y_feature = 'BirthYear'
x_feature = get_x_feature(x_f)
# print(len(x_feature))

basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-21 19:13:54.653256
Finished training
Finished prediction
Run times: 1
test dataset 有 96530 個名字, 49071 種名字
Train Accuracy ::  0.5553621772815059
Test Accuracy  ::  0.5051175800269346
平均年份絕對值誤差:: 4.832829172278048
old:: 9.434971812877064
new:: 8.78778931326793
always:: 6.398094548962856
名字只出現在共 1 個年份區間的名字比例:: 40.48 %
名字只出現在共 1 個年份區間的平均誤差:: 9.5553
random 名字只出現在共 1 個年份區間的平均誤差:: 14.9875
名字只出現在共 2 個年份區間的名字比例:: 15.229999999999999 %
名字只出現在共 2 個年份區間的平均誤差:: 4.0598
random 名字只出現在共 2 個年份區間的平均誤差:: 9.645
名字只出現在共 3 個年份區間的名字比例:: 9.83 %
名字只出現在共 3 個年份區間的平均誤差:: 2.0914
random 名字只出現在共 3 個年份區間的平均誤差:: 6.9592
名字只出現在共 4 個年份區間的名字比例:: 7.99 %
名字只出現在共 4 個年份區間的平均誤差:: 1.0635
random 名字只出現在共 4 個年份區間的平均誤差:: 5.1036
名字只出現在共 5 個年份區間的名字比例:: 6.12 %
名字只出現在共 5 個年份區間的平均誤差:: 0.5232
random 名字只出現在共 5 個年份區間的平均誤差:: 3.6339
名字只出現在共 6 個年份區間的名字比例:: 6.4399999999999995 %
名字只出現在共 6 個年份區間的平均誤差:: 0.257
random 名字只出現在共 6 個年份區間的平均誤差:: 2.6853
名字只出現在共 7 個年份區間的名字比例:: 5.27 %
名字只出現在共 7 個年份區間的平均誤差:: 0.1031
random 名字只出現在共 7 個年份區間的平均誤差:: 1.567


[30.23341862845445,
 19.42622059023528,
 13.75587646252767,
 9.981594296824369,
 7.0287745429925526,
 4.977649139733076,
 3.053708439897698,
 1.3132141810723703,
 0.47121296813862495,
 0.0]

In [172]:
%%time
# w2v + sonin 
x_f = ['w2v','sonin']

y_feature = 'BirthYear'
x_feature = get_x_feature(x_f)
# print(len(x_feature))

basic_RFClassifier_ts(reconcatedata,x_feature,y_feature,3,1,train_index,test_index,'')

2019-11-21 19:07:28.358677
Finished training
Finished prediction
Run times: 1
test dataset 有 96530 個名字, 49071 種名字
Train Accuracy ::  0.5552955801718206
Test Accuracy  ::  0.5044234952864395
平均年份絕對值誤差:: 4.855278151869885
old:: 9.576056894508099
new:: 8.739525205712665
always:: 6.351154313487242
名字只出現在共 1 個年份區間的名字比例:: 40.48 %
名字只出現在共 1 個年份區間的平均誤差:: 9.5614
random 名字只出現在共 1 個年份區間的平均誤差:: 15.0499
名字只出現在共 2 個年份區間的名字比例:: 15.229999999999999 %
名字只出現在共 2 個年份區間的平均誤差:: 4.1929
random 名字只出現在共 2 個年份區間的平均誤差:: 9.7171
名字只出現在共 3 個年份區間的名字比例:: 9.83 %
名字只出現在共 3 個年份區間的平均誤差:: 2.1202
random 名字只出現在共 3 個年份區間的平均誤差:: 6.8747
名字只出現在共 4 個年份區間的名字比例:: 7.99 %
名字只出現在共 4 個年份區間的平均誤差:: 1.0327
random 名字只出現在共 4 個年份區間的平均誤差:: 5.3133
名字只出現在共 5 個年份區間的名字比例:: 6.12 %
名字只出現在共 5 個年份區間的平均誤差:: 0.5406
random 名字只出現在共 5 個年份區間的平均誤差:: 3.5892
名字只出現在共 6 個年份區間的名字比例:: 6.4399999999999995 %
名字只出現在共 6 個年份區間的平均誤差:: 0.2476
random 名字只出現在共 6 個年份區間的平均誤差:: 2.5776
名字只出現在共 7 個年份區間的名字比例:: 5.27 %
名字只出現在共 7 個年份區間的平均誤差:: 0.0818
random 名字只出現在共 7 個年份區間的平均誤差:: 1.5

[30.03367451381781,
 19.614171086631305,
 13.817856013492147,
 10.278418664938432,
 7.220040622884225,
 5.295063515034571,
 3.1308282510328547,
 1.2680925871667155,
 0.47847959754052544,
 0.0]